# Diplomatura en Ciencia de Datos, Aprendizaje Automático y sus Aplicaciones
## Programación Distribuida sobre Grandes Volúmenes de Datos

Damián Barsotti 

### Facultad de Matemática Astronomía Física y Computación
## Universidad Nacional de Córdoba

<img src="http://program.ar/wp-content/uploads/2018/07/logo-UNC-FAMAF.png" alt="Drawing" style="width:80%;"/>


# Spark Core
---
Veremos conceptos básicos  aplicables a otras librerías de [Spark](http://spark.apache.org):
    
<img 
src="https://bitbucket.org/bigdata_famaf/diplodatos_bigdata/raw/b17129f7118b3389b8c7f2f85fd89c6238fe0edd/clases/02_spark_core/core_stack.png" alt="Drawing" style="width: 70%;"/>

# 1.- Conceptos básicos
---

### Driver

Toda aplicación Spark tiene un programa **driver**:

* lanza las operaciones en el cluster,
* contiene nuestro **programa**
    - define datos distribuidos y les aplica operaciones.

> En Zeppelin escribimos un *programa driver* que de forma interactiva ejecuta las operaciones que queremos correr.

### Executors

El driver maneja y envía tareas a **executors** en los nodos del cluster (o threads en modo local).

<img 
src="https://bitbucket.org/bigdata_famaf/diplodatos_bigdata/raw/b17129f7118b3389b8c7f2f85fd89c6238fe0edd/clases/01_intro_spark/driver_exec.png" alt="Drawing" style="width: 40%;"/>

### SparkContext

* Los programas en el driver se conectan al cluster Spark a través de un objeto `SparkContext`
* Le dice a Spark como conectarce con el cluster (o a los distintos threads en modo local)
    - (representa la conección al cluster) 
* En Zeppelin (y shell) está predefinida la variable `sc` de tipo `SparkContext`
    - otros programas deben crearla con `new`
    
<img 
src="https://bitbucket.org/bigdata_famaf/diplodatos_bigdata/raw/b17129f7118b3389b8c7f2f85fd89c6238fe0edd/clases/02_spark_core/cluster-overview.png" alt="Drawing" style="width: 60%;"/>

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("02_core").getOrCreate()
sc = spark.sparkContext

In [3]:
print(sc.defaultParallelism)
print(sc.master)

20
local[*]


|master            |descripción                                               |
|------------------|----------------------------------------------------------|
|local             |Spark corre localmente con un solo worker (no paralelismo)|
|local[K]          |Spark corre localmente con K threads                      |
|spark://HOST:PORT |se conecta a un cluster Spark                             |
|mesos://HOST:PORT |se conecta a un cluster Mesos                             |
|yarn              |se conecta a un cluster Hadoop Yarn                       |
|...               |...                                                       |



# 2.- Resilient Distributed Dataset (RDD)
---

* **Contenedores** de objetos **inmutables**, distribuidos en el cluster (contiene los datos)

* Creados con el SparkContext `sc`.
    - al cargar datasets a Spark
    - por transformaciones comunes (`map`, `filter`, ...) o binarias (`union`, `intersection`, ...).

* Ante fallas se reconstruyen (resilencia).
* **Importante**: todo lo que no derive del `SparkContext` corre solo en el **driver**.

### Ejemplo log analysis

    
<img 
src="https://bitbucket.org/bigdata_famaf/diplodatos_bigdata/raw/b17129f7118b3389b8c7f2f85fd89c6238fe0edd/clases/02_spark_core/log_linage.png" alt="Drawing" style="width: 40%;"/>

In [4]:
inputRDD = sc.textFile("../inputs/logs/") # RDD de entrada

# se crea un nuevo RDD:
errorRDD = inputRDD.filter(lambda line: "ERROR" in line) 
# se crea otro nuevo RDD
configRDD = inputRDD.filter(lambda line: "config" in line) 

errOrConfRDD = errorRDD.union(configRDD) 

for ln, l in enumerate(errOrConfRDD.collect()[:10]):
    print("Linea {}:".format(ln), l)

Linea 0: ERROR [2020-10-24 12:10:33,336] ({dispatcher-event-loop-6} Logging.scala[logError]:70) - Lost executor driver on localhost: Executor heartbeat timed out after 127728 ms
Linea 1: ERROR [2020-10-23 21:28:38,673] ({pool-2-thread-2} NewSparkInterpreter.java[open]:127) - Fail to open SparkInterpreter
Linea 2: ERROR [2020-10-23 21:28:38,673] ({pool-2-thread-2} PySparkInterpreter.java[open]:196) - Error
Linea 3: ERROR [2020-10-23 21:28:38,674] ({pool-2-thread-2} Job.java[run]:190) - Job failed
Linea 4: ERROR [2020-10-23 21:30:23,981] ({pool-2-thread-2} NewSparkInterpreter.java[open]:127) - Fail to open SparkInterpreter
Linea 5: ERROR [2020-10-23 21:30:23,981] ({pool-2-thread-2} PySparkInterpreter.java[open]:196) - Error
Linea 6: ERROR [2020-10-23 21:30:23,982] ({pool-2-thread-2} Job.java[run]:190) - Job failed
Linea 7: ERROR [2020-10-23 21:34:22,264] ({pool-2-thread-4} NewSparkInterpreter.java[open]:127) - Fail to open SparkInterpreter
Linea 8: ERROR [2020-10-23 21:34:22,264] ({pool-

### Implementación

* El RDD se distribuye en **particiones** en nodos del cluster (o fs local).
* Se construye el **grafo de operaciones**.
* Las operaciones de dividen en **tasks** (tareas).
* A cada **partición** se le aplica una **task**.
* Las tareas son ejecutadas por los executors en nodos (o threads locales).

### Ejercicio
Cree una celda nueva y copie en ella el último programa sin las líneas 13 en adelante.
Observe en Spark UI las tareas ejecutadas.

Rta:
No se Ejecuta ninguna tarea

In [5]:
inputRDD = sc.textFile("../inputs/logs/") # RDD de entrada

# se crea un nuevo RDD:
errorRDD = inputRDD.filter(lambda line: "ERROR" in line) 
# se crea otro nuevo RDD
configRDD = inputRDD.filter(lambda line: "config" in line) 
errOrConfRDD = errorRDD.union(configRDD) 

# 3.- Evaluación Lazy
---

En Spark todas las **transformaciones** (`map`, `filter`, `union`, etc.) son evaluadas de forma **lazy**:

* son acumuladas como *grafo de operaciones*
* se ejecutan al momento de traer los datos al driver (`collect`, `take`, etc.)
    - se llama a una **acción**.

Esto permite:

* hacer **optimizaciones**
    - se computa solo lo que hace falta (tiene mucho sentido en Big Data)
    - se hace un *pipeling* de transformaciones sin guardar resultados intermedios 
* recalcular las dependencias si hay algún fallo (resilencia)

### Logs análisis (muestra solo 2 lineas)
Ahora se ejecutan  

In [6]:
inputRDD = sc.textFile("../inputs/logs/") # RDD de entrada
errorRDD = inputRDD.filter(lambda line: "ERROR" in line) #  se crea un nuevo RDD
configRDD = inputRDD.filter(lambda line: "config" in line) # se crea un nuevo RDD

errOrConfRDD = errorRDD.union(configRDD) 

for ln, l in enumerate(errOrConfRDD.take(2)): # take(2) en vez de collect
    print("Linea {}:".format(ln), l)

# Compara con primer programa en Spark UI

Linea 0: ERROR [2020-10-24 12:10:33,336] ({dispatcher-event-loop-6} Logging.scala[logError]:70) - Lost executor driver on localhost: Executor heartbeat timed out after 127728 ms
Linea 1: ERROR [2020-10-23 21:28:38,673] ({pool-2-thread-2} NewSparkInterpreter.java[open]:127) - Fail to open SparkInterpreter


### Ejercicio

Complete los `...` en el siguiente programa para contar la cantidad de veces que aparece la letra 'c' en los archivos en `../inputs/logs/`.

#### Ayuda


* Se puede usar el método `.filter` (ya visto en ejemplos anteriores) para crear un RDD solo con la letra C.
* El método `count` de RDD cuenta la cantidad de elementos.
* La letra 'c' se escribe `'c'` en Scala.

In [7]:
linesRDD = sc.textFile("../inputs/logs/")

# lugar de tener una rdd de linas la idea es tener un
# rdd de characters
charsRDD = linesRDD.flatMap(lambda l: l) 
onlyCRDD = charsRDD.filter(lambda car: car=='c')

print("Aparecen {} letras c en los logs.".format(onlyCRDD.count()))

Aparecen 775948 letras c en los logs.


## 4.- Persistencia
---
Spark **recomputa** el grafo de dependencias cuando se llama una **acción**:

Entonces en lugar de reutilizar los valores, los recalcula de nuevo

In [8]:
input = sc.parallelize(range(30)) # Se crea la lista [0,...,29] y se lo convierte en RDD 

result = input.map(lambda x: x*x)

print("La media es ", result.mean()) # computa los cuadrados

for r in result.collect():
     print(r) # recomputa los cuadrados :(

La media es  285.1666666666667
0
1
4
9
16
25
36
49
64
81
100
121
144
169
196
225
256
289
324
361
400
441
484
529
576
625
676
729
784
841


## 5.- Implementación API Python
---

Spark paraleliza tambien la laectura, ya que cada thread va a a leer una parte
distita de un archivo.

* Spark esta originalmente implementado en Scala/Java.
* `SparkContext` de Python usa [Py4J](https://www.py4j.org/), lanza JVM local y crea `JavaSparkContext`.
* [Py4J](https://www.py4j.org/) solo se usa en driver.
* En máquinas remotas los executors corren en JVM asegurando resilencia.
* Estas JVM lanzan procesos Python.
* [Mas info](https://medium.com/@ketanvatsalya/a-scenic-route-through-pyspark-internals-feaf74ed660d).


    
<img 
src="https://bitbucket.org/bigdata_famaf/diplodatos_bigdata/raw/b17129f7118b3389b8c7f2f85fd89c6238fe0edd/clases/02_spark_core/python-spark.png" alt="Drawing" style="width: 60%;"/>


### Ejercicio

Complete el siguiente programa para que cuente la cantidad de lineas que comienzan con la palabra `INFO`, `WARN` y `ERROR`.

También, haga cache de los RDD para hacer el programa más eficiente. 

Cuando se ponde `chache` se le dice a los nodos que guarden la parte de rrdd
que tiene cada nodo

In [9]:
linesRDD = sc.textFile("../inputs/logs/") # RDD de entrada
linesStrip = linesRDD.map(lambda l: l.strip()).cache() # Borro espacios en borde
linesInfo = linesStrip.filter(lambda l: l.startswith("INFO"))
linesWarn = linesStrip.filter(lambda l: l.startswith("WARN"))
linesError = linesStrip.filter(lambda l: l.startswith("ERROR"))

print("Cantidad de lineas INFO: ", linesInfo.count())

print("Cantidad de lineas WARN: ", linesWarn.count()) #Completar

print("Cantidad de lineas ERROR: ", linesError.count())  #Completar

Cantidad de lineas INFO:  205710
Cantidad de lineas WARN:  1690
Cantidad de lineas ERROR:  104


### Ejercicio

El archivo en `../inputs/ds/flights.csv` contiene información de vuelos realizados en 2008 (solo 100.000), uno por línea.

Los datos estan separados por coma y la columna `Cancelled` (la 22) tiene un `1` si el vuelo fue cancelado. Además si el vuelo fue redirigido se indica con '1' en la columna `Diverted` (la 24).

Completar el siguiente programa que devuelve el porcentaje de vuelos cancelados y el porcentaje de redirigidos.

Utilizar cache si lo cree conveniente.

In [10]:
df = spark.read.format("csv")\
        .option("header", "true")\
        .load("../inputs/ds/flights.csv")\
        .sample(False, 0.0005)
df.printSchema()

root
 |-- Year: string (nullable = true)
 |-- Month: string (nullable = true)
 |-- DayofMonth: string (nullable = true)
 |-- DayOfWeek: string (nullable = true)
 |-- DepTime: string (nullable = true)
 |-- CRSDepTime: string (nullable = true)
 |-- ArrTime: string (nullable = true)
 |-- CRSArrTime: string (nullable = true)
 |-- UniqueCarrier: string (nullable = true)
 |-- FlightNum: string (nullable = true)
 |-- TailNum: string (nullable = true)
 |-- ActualElapsedTime: string (nullable = true)
 |-- CRSElapsedTime: string (nullable = true)
 |-- AirTime: string (nullable = true)
 |-- ArrDelay: string (nullable = true)
 |-- DepDelay: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Distance: string (nullable = true)
 |-- TaxiIn: string (nullable = true)
 |-- TaxiOut: string (nullable = true)
 |-- Cancelled: string (nullable = true)
 |-- CancellationCode: string (nullable = true)
 |-- Diverted: string (nullable = true)
 |-- CarrierDelay:

In [11]:
input = sc.textFile("../inputs/ds/flights.csv") # Completar el path

nTotal = input.count() - 1 # la primer fila tiene el nombre de las columnas
parsed = input.map(lambda l: l.split(",")).cache()
cancel = parsed.filter(lambda l: l[21] == '1') # Completar
redir = parsed.filter(lambda l: l[23] == '1') # Completar

nCancel = cancel.count()#.cache()
nRedir = redir.count() #.cache()

print("cancelados = {}%".format(float(nCancel) * 100 / nTotal))
print("redireccionados = {}%".format(float(nRedir) * 100 / nTotal)) # Completar

cancelados = 1.142%
redireccionados = 0.16%


### Ejercicio

La columna 14 del mismo archivo tiene el tiempo del vuelo en minutos. Calcular el máximo.

#### Ayuda

* Busque en la documentacion de la [API RDD](http://spark.apache.org/docs/2.2.1/api/python/pyspark.html#pyspark.RDD) una acción para calcular el máximo.
* Ojo que puede haber valores no definidos.

In [12]:
sc.textFile("../inputs/ds/flights.csv") \
    .map(lambda line: line.split(",")) \
    .filter(lambda line: line[13].isdigit() ) \
    .max()[13]

'126'

Fin